In [3]:
import findspark
#useful to install this tool to simplify spark import
findspark.init()
from pyspark import  SparkContext
sc = SparkContext( 'local', 'pyspark')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark, master=local) created by __init__ at <ipython-input-2-74f8aacb7141>:5 

In [84]:
import numpy
rdd=sc.textFile('train_no_header.csv')

#obtaining rdd: itemid,rating (users are not useful for this recommendation)
def firstMap (line):
    x=line.split(',')
    return x[1],int(x[2])
item_rdd = rdd.map(firstMap).cache()

#computing average vote of the entire dataset
totalVotes = item_rdd.map(lambda x: x[1]).reduce(lambda x,y: x+y)
avgVote = float(totalVotes)/rdd.count()

#creating rdd: itemid, [rating_list]
itemVotes=item_rdd.groupByKey()
itemVotes=itemVotes.map(lambda x:(x[0],list(x[1])))
itemAverage=itemVotes.map(lambda x: (x[0],float(sum(x[1]))/len(x[1]),len(x[1]))).cache()

#computing average number of vote for item of the entire dataset
avgVoteNumberPerItem=item_rdd.count()/itemVotes.count()


In [102]:
#Computing ranking value according to:
#https://www.quora.com/How-is-a-Movies-weighted-rating-calculated-on-IMDb
minimumVotesRequired=avgVoteNumberPerItem*4

def tenStarScore (x):
    R=x[1]
    v=x[2]
    m=minimumVotesRequired
    C=avgVote
    return x[0],(v*R+m*C)/(v+m),x[2]
    
itemRank=itemAverage.map(tenStarScore)            

print minimumVotesRequired
top250=itemRank.takeOrdered(250,key=lambda x: -x[1])
print top250

20
[(u'33475', 9.112799216645667, 72), (u'33173', 9.030189847802768, 125), (u'15743', 8.78292134664633, 75), (u'32347', 8.778303903945254, 64), (u'36872', 8.766778638468056, 77), (u'1076', 8.762754888253717, 88), (u'34517', 8.740412394850564, 71), (u'29848', 8.739488288330136, 63), (u'3431', 8.718780094421444, 64), (u'35300', 8.718516487247165, 103), (u'12866', 8.701674791805672, 86), (u'17648', 8.701674791805672, 86), (u'29915', 8.696018863350634, 61), (u'25363', 8.69497057061192, 64), (u'13502', 8.679719099142517, 60), (u'9577', 8.627158358388046, 97), (u'31992', 8.618394224711267, 131), (u'11259', 8.614606740105568, 88), (u'18200', 8.604342315273694, 113), (u'6461', 8.599802876591239, 94), (u'33175', 8.579719099142517, 60), (u'10446', 8.573487087117556, 94), (u'2041', 8.572471908311378, 68), (u'27569', 8.564588219026389, 71), (u'23188', 8.532331774510128, 69), (u'24639', 8.521931999554555, 134), (u'2911', 8.509643164081334, 71), (u'14616', 8.508050714966982, 89), (u'2049', 8.4861423

In [103]:
rdd=sc.textFile('train_no_header.csv')
rdd = rdd.map( lambda x: x.split(',') )
rdd = rdd.map(lambda x:(x[0],[x[1],x[2]]))

rdd = rdd.groupByKey()
URM = rdd.map(lambda x:(x[0],list(x[1])))
URM.takeOrdered(10)




[(u'1',
  [[u'3456', u'7'],
   [u'3527', u'10'],
   [u'3864', u'8'],
   [u'3920', u'10'],
   [u'5248', u'10'],
   [u'6111', u'10'],
   [u'6249', u'7'],
   [u'6703', u'8'],
   [u'7393', u'9'],
   [u'11618', u'1'],
   [u'14227', u'5'],
   [u'16571', u'10'],
   [u'19022', u'9'],
   [u'19478', u'4'],
   [u'19629', u'8'],
   [u'20484', u'9'],
   [u'21101', u'10'],
   [u'21195', u'2'],
   [u'22932', u'9'],
   [u'23507', u'10'],
   [u'25533', u'6'],
   [u'27660', u'10'],
   [u'28083', u'10'],
   [u'30478', u'3'],
   [u'33173', u'10'],
   [u'33213', u'9'],
   [u'33603', u'10']]),
 (u'10', [[u'17463', u'5'], [u'21660', u'9']]),
 (u'100', [[u'8013', u'9'], [u'12255', u'2'], [u'30796', u'10']]),
 (u'1000', [[u'3545', u'9'], [u'15593', u'9'], [u'29915', u'10']]),
 (u'10000',
  [[u'2339', u'10'],
   [u'4926', u'10'],
   [u'7992', u'3'],
   [u'9012', u'10'],
   [u'13998', u'10'],
   [u'14532', u'2'],
   [u'17207', u'1'],
   [u'17988', u'1'],
   [u'18669', u'3'],
   [u'19907', u'2'],
   [u'21101', u'

In [104]:
test_users=sc.textFile('test_no_header.csv')

test_list=test_users.collect()

def filter_test_users(x):
    return x[0] in test_list

test_URM=URM.filter(filter_test_users)

print test_URM.count()

test_URM.take(10)

def top_recommendations(x):
    y=[]

    for item in top250:
        if item[0] not in x[1]:
            y.append(item[0])
        else:
            print "ALLELUIA"
        if len(y)>=5: 
            return x[0],y
    return x[0],y
        
recommendations=test_URM.map(top_recommendations)
recommendations.take(10)

def map_string (x):
    y=x[1]
    s=y[0]+" "+y[1]+" "+y[2]+" "+y[3]+" "+y[4]
    return x[0],s
formatted_recommendations=recommendations.map(map_string).takeOrdered(4196,key=lambda x: int(x[0]))
len(formatted_recommendations)


4196


4196

In [105]:
import csv
f1= open('submission1.csv','w+')

csv_f1=csv.writer(f1)
csv_f1.writerow(["userId","testItems"])
for i in formatted_recommendations:
    csv_f1.writerow(i)